<a href="https://colab.research.google.com/github/chinmayj23/ML/blob/master/ML_remedial1_J079.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Credit-approval

In [71]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn import linear_model
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import warnings
warnings.filterwarnings('ignore')

np.random.seed(27)

In [72]:
data=pd.read_csv("/content/drive/My Drive/dataset_31_credit-g.csv")

In [73]:
data.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,'<0',6,'critical/other existing credit',radio/tv,1169,'no known savings','>=7',4,'male single',none,4,'real estate',67,none,own,2,skilled,1,yes,yes,good
1,'0<=X<200',48,'existing paid',radio/tv,5951,'<100','1<=X<4',2,'female div/dep/mar',none,2,'real estate',22,none,own,1,skilled,1,none,yes,bad
2,'no checking',12,'critical/other existing credit',education,2096,'<100','4<=X<7',2,'male single',none,3,'real estate',49,none,own,1,'unskilled resident',2,none,yes,good
3,'<0',42,'existing paid',furniture/equipment,7882,'<100','4<=X<7',2,'male single',guarantor,4,'life insurance',45,none,'for free',1,skilled,2,none,yes,good
4,'<0',24,'delayed previously','new car',4870,'<100','1<=X<4',3,'male single',none,4,'no known property',53,none,'for free',2,skilled,2,none,yes,bad


In [74]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   checking_status         1000 non-null   object
 1   duration                1000 non-null   int64 
 2   credit_history          1000 non-null   object
 3   purpose                 1000 non-null   object
 4   credit_amount           1000 non-null   int64 
 5   savings_status          1000 non-null   object
 6   employment              1000 non-null   object
 7   installment_commitment  1000 non-null   int64 
 8   personal_status         1000 non-null   object
 9   other_parties           1000 non-null   object
 10  residence_since         1000 non-null   int64 
 11  property_magnitude      1000 non-null   object
 12  age                     1000 non-null   int64 
 13  other_payment_plans     1000 non-null   object
 14  housing                 1000 non-null   object
 15  exist

In [75]:
y=data['class']
data.drop(columns=['class'],inplace=True,axis=1,index=None)

In [76]:
df=data[['duration','credit_amount','installment_commitment','residence_since','age','existing_credits','num_dependents']]

In [77]:
df.head()

,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents
0,6,1169,4,4,67,2,1
1,48,5951,2,2,22,1,1
2,12,2096,2,3,49,1,2
3,42,7882,2,4,45,1,2
4,24,4870,3,4,53,2,2


In [78]:
foreign_worker=pd.get_dummies(data['foreign_worker'])
own_telephone=pd.get_dummies(data['own_telephone'])
job=pd.get_dummies(data['job'])
housing=pd.get_dummies(data['housing'])
other_payment_plans=pd.get_dummies(data['other_payment_plans'])
property_magnitude=pd.get_dummies(data['property_magnitude'])
other_parties=pd.get_dummies(data['other_parties'])
personal_status=pd.get_dummies(data['personal_status'])
employment=pd.get_dummies(data['employment'])
savings_status=pd.get_dummies(data['savings_status'])
purpose=pd.get_dummies(data['purpose'])
credit_history=pd.get_dummies(data['credit_history'])
checking_status=pd.get_dummies(data['checking_status'])

In [79]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y.unique())
y = le.transform(y)

In [80]:
frames=[df,foreign_worker,own_telephone,job,housing,other_payment_plans,property_magnitude,other_parties,personal_status,employment,savings_status,purpose,credit_history,checking_status]

In [81]:
x=pd.concat(frames,axis=1)

In [82]:
x.head()

,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents,no,yes,none,yes,'high qualif/self emp/mgmt','unemp/unskilled non res','unskilled resident',skilled,'for free',own,rent,bank,none,stores,'life insurance','no known property','real estate',car,'co applicant',guarantor,none,'female div/dep/mar','male div/sep','male mar/wid','male single','1<=X<4','4<=X<7','<1','>=7',unemployed,'100<=X<500','500<=X<1000','<100','>=1000','no known savings','domestic appliance','new car','used car',business,education,furniture/equipment,other,radio/tv,repairs,retraining,'all paid','critical/other existing credit','delayed previously','existing paid','no credits/all paid','0<=X<200','<0','>=200','no checking'
0,6,1169,4,4,67,2,1,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0
1,48,5951,2,2,22,1,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0
2,12,2096,2,3,49,1,2,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1
3,42,7882,2,4,45,1,2,0,1,1,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0
4,24,4870,3,4,53,2,2,0,1,1,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0


In [83]:
seed = 8
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=seed)

In [84]:
logistic = linear_model.LogisticRegression()

In [85]:
logistic.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [86]:
penalty = ['l1', 'l2']
C = [0.0001, 0.001, 0.01]
class_weight = [{1:0.5, 0:0.5}, {1:0.6, 0:0.4}]
solver = ['liblinear', 'saga'] #

paramgrid = dict(penalty=penalty, C=C, class_weight=class_weight, solver=solver)

random = RandomizedSearchCV(estimator=logistic, param_distributions=paramgrid, scoring='roc_auc', verbose=1, n_jobs=-1)
random_result = random.fit(X_train, y_train)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    1.4s finished


In [89]:
random_result.get_params

<bound method BaseEstimator.get_params of RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=LogisticRegression(C=1.0, class_weight=None,
                                                dual=False, fit_intercept=True,
                                                intercept_scaling=1,
                                                l1_ratio=None, max_iter=100,
                                                multi_class='auto', n_jobs=None,
                                                penalty='l2', random_state=None,
                                                solver='lbfgs', tol=0.0001,
                                                verbose=0, warm_start=False),
                   iid='deprecated', n_iter=10, n_jobs=-1,
                   param_distributions={'C': [0.0001, 0.001, 0.01],
                                        'class_weight': [{0: 0.5, 1: 0.5},
                                                         {0: 0.4, 1: 0.6}],
                   

In [90]:
random_result.best_params_

{'C': 0.01,
 'class_weight': {0: 0.5, 1: 0.5},
 'penalty': 'l2',
 'solver': 'liblinear'}

In [91]:
y_pred1 = random_result.predict(X_test)

In [94]:
from sklearn.metrics import accuracy_score
acc=accuracy_score(y_test,y_pred1)
acc

0.715